# GGA 核坐标梯度性质

在这一节中，我们将会进一步拓展核坐标梯度性质的学习，将 GGA 引入 SCF，获得 GGA 的一阶核梯度与二阶核梯度的性质．这里所指的 GGA 是以 GGA 为基础的泛函，包括杂化的与非杂化的．双杂化的 GGA 将会在以后讨论．

一般来说，近 30 年的中小规模分子量化计算的主流始终是 DFT 近似．这源于 DFT 近似的计算量与 HF 相当，但结果通常比 HF 优异许多，可以媲美最低阶的 Post-HF 方法 MP2 的结果．

DFT 近似各式各样；一般来说，化学领域的最主流是 B3LYP，而物理领域的最主流是 PBE；这两者都是以 GGA 为基础的泛函．在这一节中，我们会以 B3LYP 为例，了解运用 PySCF 的底层函数来解决 B3LYP 的一阶梯度与二阶梯度．这里指的 PySCF 底层函数是指求取 DFT 积分格点的函数．

关于 B3LYP 的能量的计算与调用细节，可以前往 [XYG3 能量](xyg3_energy.ipynb#输出-2：交换相关能格点求取) 文档的其中一小节．这一节的记号会稍有不同，但应当容易理解．许多基础的公式与程序细节这一节不再叙述．

In [1]:
from pyscf import gto, scf, dft, grad, data, lib, hessian
import numpy as np

import pyscf.grad.rks
import pyscf.hessian.rks
np.set_printoptions(5, linewidth=150, suppress=True)

## 准备工作

### 顶层函数计算 RKS 梯度

In [2]:
mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.0  0.0  0.0
H  0.0  1.0  1.5
"""
mol.basis = "6-31G"
mol.build()

In [3]:
scf_eng = dft.RKS(mol)
scf_eng.xc = "b3lypg"  # compare that to gaussian
scf_eng.grids.atom_grid = (99, 590)
scf_eng.grids.build()
scf_eng.conv_tol = 1e-13
energy_RKS = scf_eng.kernel()

converged SCF energy = -151.477320518606


In [4]:
scf_grad = grad.rks.Gradients(scf_eng)
grad_RKS = scf_grad.kernel()

--------------- RKS gradients ---------------
         x                y                z
0 O    -0.0271975153     0.0107158349     0.0176473933
1 O     0.0107158349    -0.0271975153    -0.0176473933
2 H     0.0099981076     0.0064834995     0.0257989619
3 H     0.0064834995     0.0099981076    -0.0257989619
----------------------------------------------


In [5]:
scf_hess = hessian.rks.Hessian(scf_eng)
hess_RKS = scf_hess.kernel()

### 与 Gaussian 结果进行比对

In [6]:
def val_from_fchk(key, file_path):
    flag_read = False; expect_size = -1; vec = []
    with open(file_path, "r") as file:
        for l in file:
            if (l[:len(key)] == key):
                try: expect_size = int(l[len(key):].split()[2]); flag_read = True; continue
                except IndexError: return float(l[len(key):].split()[1])
            if (flag_read):
                try: vec += [ float(i) for i in l.split() ]
                except ValueError: break
    if len(vec) != expect_size: raise ValueError("Number of expected size is not consistent with read-in size!")
    return np.array(vec)

In [7]:
energy_Gaussian = val_from_fchk("SCF Energy", "include/B3LYP-hess.fch")
grad_Gaussian = val_from_fchk("Cartesian Gradient", "include/B3LYP-hess.fch")
hess_Gaussian = val_from_fchk("Cartesian Force Constants", "include/B3LYP-hess.fch")

由于 PySCF 的 DFT 积分与 Gaussian 仍然似乎略有不同，因此尽管数据大体能对上，但随着导数的阶越大，偏差也越大；二阶梯度在这两个程序的匹配精度只有 $10^{-4}$ 级别．

In [8]:
print(np.allclose(energy_RKS, energy_Gaussian))
print(np.allclose(grad_RKS.reshape(-1), grad_Gaussian, atol=1e-5))
d_hess = mol.natm * 3
print(np.allclose(hess_RKS.swapaxes(1, 2).reshape(d_hess, d_hess)[np.tril_indices(d_hess)], hess_Gaussian, atol=1e-4))

True
True
True


### RKS 重要中间量

In [9]:
c_x = scf_eng._numint.hybrid_coeff(scf_eng.xc)

nao = mol.nao
nmo = scf_eng.mo_energy.shape[0]
nelec = mol.nelectron
nocc = mol.nelec[0]
nvir = nmo - nocc

C = scf_eng.mo_coeff
Co = C[:, :nocc]
Cv = C[:, nocc:]
e = scf_eng.mo_energy
eo = e[:nocc]
ev = e[nocc:]
mo_occ = scf_eng.mo_occ

D = scf_eng.make_rdm1()
De = np.einsum("ui, i, vi -> uv", C, e * mo_occ, C)

In [10]:
natm = mol.natm

# grad-contrib
int1e_ipovlp = mol.intor('int1e_ipovlp')
int1e_ipkin = mol.intor("int1e_ipkin")
int1e_ipnuc = mol.intor("int1e_ipnuc")
int2e_ip1 = mol.intor("int2e_ip1")

# hess-contrib
int1e_ipipkin = mol.intor("int1e_ipipkin")
int1e_ipkinip = mol.intor("int1e_ipkinip")
int1e_ipipnuc = mol.intor("int1e_ipipnuc")
int1e_ipnucip = mol.intor("int1e_ipnucip")
int2e_ipip1 = mol.intor("int2e_ipip1")
int2e_ipvip1 = mol.intor("int2e_ipvip1")
int2e_ip1ip2 = mol.intor("int2e_ip1ip2")
int1e_ipipovlp = mol.intor("int1e_ipipovlp")
int1e_ipovlpip = mol.intor("int1e_ipovlpip")

In [11]:
def mol_slice(atm_id):
    _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
    return slice(p0, p1)

## GGA 一阶核坐标梯度

### HF 部分

GGA 的一阶梯度中，大部分仍然是 HF 的贡献．我们在以前已经讨论过 [HF 梯度](hf_nuc_grad.ipynb)；GGA 中，至多就是在交换积分的部分乘上杂化泛函的系数．对于 B3LYP，$c_x = 0.2$．

In [12]:
def my_grad_hf(scf_eng):
    mol = scf_eng.mol
    C = scf_eng.mo_coeff
    D = scf_eng.make_rdm1()
    mo_occ = scf_eng.mo_occ
    e = scf_eng.mo_energy
    De = np.einsum("ui, i, vi -> uv", C, e * mo_occ, C)
    c_x = scf_eng._numint.hybrid_coeff(scf_eng.xc)
    integrals  = - mol.intor("int1e_ipkin") - mol.intor("int1e_ipnuc") \
                 - np.einsum("tuvkl, kl -> tuv", mol.intor('int2e_ip1'), D) \
                 + c_x * 0.5 * np.einsum("tukvl, kl -> tuv", mol.intor('int2e_ip1'), D)
    integral_ovlp = mol.intor('int1e_ipovlp')
    grad_elec = np.zeros((mol.natm, 3))
    for A in range(mol.natm):
        sA = mol_slice(A)
        grad_elec[A, :] += np.einsum("tuv, uv -> t", integrals[:, sA], D[sA]) * 2
        grad_elec[A, :] += np.einsum("tuv, uv -> t", integral_ovlp[:, sA], De[sA]) * 2
        with mol.with_rinv_as_nucleus(A):
            grad_elec[A, :] -= np.einsum("tuv, uv -> t", mol.intor("int1e_iprinv") * mol.atom_charge(A), D) * 2
    return grad_elec

grad_hf = my_grad_hf(scf_eng)

### GGA 部分

GGA 电子态能量一阶梯度的另一个贡献的部分是交换相关能关于坐标的导数．能量导数的表达式的产生并非显然；下面简单叙述导数表达式的产生过程．

首先，根据 [泛函变分的意义](https://en.wikipedia.org/wiki/Functional_derivative#Functional_derivative)，有下式成立：

\begin{equation}
\int \frac{\delta E_\mathrm{xc}}{\rho(\boldsymbol{r})} f(\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r} = \lim_{\varepsilon \rightarrow 0} \frac{E_\mathrm{xc} [\rho + \varepsilon f] - E_\mathrm{xc} [\rho]}{\varepsilon} = \left. \frac{\partial}{\partial t} E_\mathrm{xc} [\rho + \varepsilon f] \right|_{\varepsilon \rightarrow 0}
\tag{1} \label{eq.1}
\end{equation}

其次，我们从微元法的角度指出，如果现在 $\varepsilon$ 指代原子核坐标分量的移动量 $A_t$；$\rho$ 是未被微扰的密度，而 $\rho + \varepsilon f$ 是沿着 $\varepsilon$ 所指代的方向被微扰后的电子密度，那么微扰密度将是 $f(\boldsymbol{r}) = \partial_{A_t} \rho = D_{\mu \nu} \partial_{A_t} (\phi_\mu \phi_\nu)$．我们将这个结果代入上式，则有

\begin{equation}
\left. \frac{\partial}{\partial A_t} E_\mathrm{xc} [\rho + A_t \partial_{A_t} \rho] \right|_{A_t \rightarrow 0} = \int \frac{\delta E_\mathrm{xc}}{\rho(\boldsymbol{r})} \frac{\partial \rho(\boldsymbol{r})}{\partial A_t} \, \mathrm{d} \boldsymbol{r}
\end{equation}

我们就认为上式即是交换相关能对原子核坐标分量 $A_t$ 的导数 $\partial_{A_t} E_\mathrm{xc} [\rho]$ 了．因此，上式经过变分展开后可以写为

\begin{align}
\partial_{A_t} E_\mathrm{xc}[\rho]
&= \int \Big( \partial_\rho f^\mathrm{xc} \cdot \partial_{A_t} \rho + 2 \partial_\gamma f^\mathrm{xc} \cdot \partial_s \rho \cdot \partial_s (\partial_{A_t} \rho) \Big) \, \mathrm{d} \boldsymbol{r} \\
&= D_{\mu \nu} \int \Big( \partial_\rho f^\mathrm{xc} \cdot (\partial_{A_t} \phi_\mu \cdot \phi_\nu + \phi_\mu \cdot \partial_{A_t} \phi_\nu) \\
&\quad + 2 \partial_\gamma f^\mathrm{xc} \cdot \partial_s \rho \cdot \Big(
\partial_{A_t} \partial_s \phi_\mu \cdot \phi_\nu + \phi_\mu \cdot \partial_{A_t} \partial_s \phi_\nu +
\partial_{A_t} \phi_\mu \cdot \partial_s \phi_\nu + \partial_s \phi_\mu \cdot \partial_{A_t} \phi_\nu
\Big)
\, \mathrm{d} \boldsymbol{r} \\
&= D_{\mu \nu} w_g \cdot \partial_\rho f^\mathrm{xc}_g \cdot (\partial_{A_t} \phi_{g \mu} \cdot \phi_{g \nu} + \phi_{g \mu} \cdot \partial_{A_t} \phi_{g \nu}) \\
&\quad + 2 D_{\mu \nu} w_g \cdot \partial_\gamma f^\mathrm{xc}_g \cdot \partial_s \rho_g \cdot \Big(
\partial_{A_t} \partial_s \phi_{g \mu} \cdot \phi_{g \nu} + \phi_{g \mu} \cdot \partial_{A_t} \partial_s \phi_{g \nu} +
\partial_{A_t} \phi_{g \mu} \cdot \partial_s \phi_{g \nu} + \partial_s \phi_{g \mu} \cdot \partial_{A_t} \phi_{g \nu}
\Big) \\
&= 2 D_{\mu \nu} w_g \cdot \partial_\rho f^\mathrm{xc}_g \cdot \partial_{A_t} \phi_{g \mu} \cdot \phi_{g \nu} +
4 D_{\mu \nu} w_g \cdot \partial_\gamma f^\mathrm{xc}_g \cdot \partial_s \rho_g \cdot \partial_{A_t} \partial_s \phi_{g \mu} \cdot \phi_{g \nu} +
4 D_{\mu \nu} w_g \cdot \partial_\gamma f^\mathrm{xc}_g \cdot \partial_s \rho_g \cdot \partial_{A_t} \phi_{g \mu} \cdot \partial_s \phi_{g \nu} \\
&= - D_{\mu_A \nu} \left( 2 w_g \cdot \partial_\rho f^\mathrm{xc}_g \cdot \partial_t \phi_{g \mu_A} \cdot \phi_{g \nu} +
4 w_g \cdot \partial_\gamma f^\mathrm{xc}_g \cdot \partial_s \rho_g \cdot \partial_t \partial_s \phi_{g \mu_A} \cdot \phi_{g \nu} +
4 w_g \cdot \partial_\gamma f^\mathrm{xc}_g \cdot \partial_s \rho_g \cdot \partial_t \phi_{g \mu_A} \cdot \partial_s \phi_{g \nu} \right)
\tag{2} \label{eq.2}
\end{align}

其中，上式的第一个等号是泛函变分的定义；第二个等号是将 $\rho(\boldsymbol{r}) = D_{\mu \nu} \phi_{\mu}(\boldsymbol{r}) \phi_{\nu}(\boldsymbol{r})$ 代入；第三个等号是将积分转换为格点求和；第四个等号是借用对 $\mu, \nu$ 求和时，两个角标可以互换而简化一半的项；第五个等号则是利用 $\partial_{A_t} \phi_{\mu} = - \partial_t \phi_{\mu_A}$，即对原子核坐标的导数等于对电子导数的负值，但只对当前原子核上的基轨道求导才有意义．

由于后面将会大量碰到对轨道 $\phi_{\mu}$、泛函核 $f_{g}^\mathrm{xc}$ 的偏导数，记号会变得非常麻烦．为了简记记号，让公式看起来更简洁，但又不影响公式的意义，以及公式到程序的对应，我们规定以下记号及其衍生记号：

* $\phi_{tsrg \mu}$ = $\partial_t \partial_s \partial_r \phi_{g \mu}$；以后，$t, s, r, w$ 放在下标时表示三维坐标分量；

* $f_{\rho \gamma g}$ = $\partial_\rho \partial_\gamma f_{g}^\mathrm{xc}$；

那么式 [(2)](#mjx-eqn-eq.2) 可以简记为

\begin{equation}
\partial_{A_t} E_\mathrm{xc}[\rho] = - D_{\mu_A \nu} \big(
2 w_g f_{\rho g} \phi_{t g \mu_A} \phi_{g \nu} + 4 w_g f_{\gamma g} \rho_{s g} \phi_{tsg \mu_A} \phi_{g \nu} + 4 w_g f_{\gamma g} \rho_{sg} \phi_{tg \mu_A} \phi_{s \nu}
\big)
\tag{3} \label{eq.3}
\end{equation}

简单的代码展示如下．完整执行上述公式的部分是第 (4) 与 (5) 部分．前 (3) 部分基本可以参考 [之前的文档](xyg3_energy.ipynb#输出-2：交换相关能格点求取)；不过以前只是求了 GGA 的 Fock 矩阵，最多需要基轨道的一阶导数 $\partial_t \phi_{g \mu}$；但在 GGA 梯度的任务里，则需要基轨道的二阶导数 $\partial_t \partial_s \phi_{g \nu}$．这部分的生成中，需要将 `scf_eng._numint.block_loop` 传入的第 4 个参数改为 2，以表示需要二阶导数信息；同时新生成了 `grid_ao_2` 储存这些二阶导数基轨道格点．

In [13]:
def my_grad_gga(scf_eng):
    mol = scf_eng.mol
    D = scf_eng.make_rdm1()
    # (1) generate \phi_{g \mu}, \partial_{t} \phi_{g \mu}, \partial_{ts} \phi_{g \mu}
    grid_ao, grid_mask, grid_weight, grid_coords = next(scf_eng._numint.block_loop(mol, scf_eng.grids, nao, 2, 2000))
    ngrid = grid_ao.shape[1]
    grid_ao_0 = grid_ao[0]
    grid_ao_1 = grid_ao[1:4]
    grid_ao_2 = [
        [ grid_ao[4], grid_ao[5], grid_ao[6] ],
        [ grid_ao[5], grid_ao[7], grid_ao[8] ],
        [ grid_ao[6], grid_ao[8], grid_ao[9] ],
    ]
    # (2) generate \rho_g, \partial_s \rho_g
    grid_rho = np.einsum("uv, tgu, gv -> tg", D, grid_ao[0:4], grid_ao_0, optimize=True)
    grid_rho[1:] *= 2
    grid_rho_0 = grid_rho[0]
    grid_rho_1 = grid_rho[1:4]
    # (3) generate \partial_\rho f_g^\mathrm{xc}, \partial_\gamma f_g^\mathrm{xc}
    grid_fr, grid_fg = scf_eng._numint.eval_xc(scf_eng.xc, grid_rho, deriv=1)[1][0:2]
    # (4) generate atomic basis part of xc derivative
    integrals = np.zeros((3, nao, nao))
    integrals -= 2 * np.einsum("g, g, tgu, gv -> tuv",      grid_weight, grid_fr, grid_ao_1,  grid_ao_0,            optimize=True)
    integrals -= 4 * np.einsum("g, g, sg, sgv, tgu -> tuv", grid_weight, grid_fg, grid_rho_1, grid_ao_1, grid_ao_1, optimize=True)
    integrals -= 4 * np.einsum("g, g, sg, tsgu, gv -> tuv", grid_weight, grid_fg, grid_rho_1, grid_ao_2, grid_ao_0, optimize=True)
    # (5) summation by different atoms
    grad_elec = np.zeros((mol.natm, 3))
    for atm_id in range(mol.natm):
        _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
        grad_elec[atm_id, :] += np.einsum("tuv, uv -> t", integrals[:, p0:p1], D[p0:p1])
    return grad_elec

grad_gga = my_grad_gga(scf_eng)

GGA 的一阶梯度的所有部分都求取完毕．我们只需要将其与 HF 部分相加，就可以得到总的 GGA 电子能量梯度．

In [14]:
np.allclose(grad_hf + grad_gga, scf_grad.grad_elec())

True

## GGA 二阶梯度

In [15]:
grid_ao, grid_mask, grid_weight, grid_coords = next(scf_eng._numint.block_loop(mol, scf_eng.grids, nao, 3, 2000))
ngrid = grid_ao.shape[1]

In [16]:
grid_ao_0  = grid_ao[0]
grid_ao_1  = grid_ao[1:4]
grid_ao_2T = grid_ao[4:10]

XX, XY, XZ, YY, YZ, ZZ = range(4, 10)
XXX, XXY, XXZ, XYY, XYZ, XZZ, YYY, YYZ, YZZ, ZZZ = range(10, 20)

grid_ao_2 = np.array([
    [grid_ao[XX], grid_ao[XY], grid_ao[XZ]],
    [grid_ao[XY], grid_ao[YY], grid_ao[YZ]],
    [grid_ao[XZ], grid_ao[YZ], grid_ao[ZZ]],
])
grid_ao_3T = np.array([
    [grid_ao[XXX], grid_ao[XXY], grid_ao[XXZ], grid_ao[XYY], grid_ao[XYZ], grid_ao[XZZ]],
    [grid_ao[XXY], grid_ao[XYY], grid_ao[XYZ], grid_ao[YYY], grid_ao[YYZ], grid_ao[YZZ]],
    [grid_ao[XXZ], grid_ao[XYZ], grid_ao[XZZ], grid_ao[YYZ], grid_ao[YZZ], grid_ao[ZZZ]],
])

In [17]:
grid_rho = np.einsum("uv, tgu, gv -> tg", D, grid_ao[0:4], grid_ao_0, optimize=True)
grid_rho[1:] *= 2
grid_rho_0 = grid_rho[0]
grid_rho_1 = grid_rho[1:4]

In [18]:
grid_vxc, grid_fxc = scf_eng._numint.eval_xc(scf_eng.xc, grid_rho, deriv=2)[1:3]
grid_fr, grid_fg = grid_vxc[0:2]
grid_frr, grid_frg, grid_fgg = grid_fxc[0:3]

### 同核部分

In [19]:
vxc_diag  = np.einsum("g, g, Tgu, gv -> Tuv", grid_weight, grid_fr, grid_ao_2T, grid_ao_0, optimize=True)
vxc_diag += 2 * np.einsum("g, g, rg, rTgu, gv -> Tuv", grid_weight, grid_fg, grid_rho_1, grid_ao_3T, grid_ao_0, optimize=True)
vxc_diag += 2 * np.einsum("g, g, rg, Tgu, rgv -> Tuv", grid_weight, grid_fg, grid_rho_1, grid_ao_2T, grid_ao_1, optimize=True)
XX, XY, XZ, YY, YZ, ZZ = range(6)
vxc_diag = vxc_diag[[XX, XY, XZ, XY, YY, YZ, XZ, YZ, ZZ]]
vxc_diag.shape = (3, 3, nao, nao)

In [20]:
vxc_diag_pyscf = hessian.rks._get_vxc_diag(scf_hess, C, mo_occ, 2000)

In [21]:
np.allclose(vxc_diag_pyscf, vxc_diag)

True

In [22]:
hess_veff_diag = np.zeros((natm, natm, 3, 3))
for A in range(natm):
    sA = mol_slice(A)
    hess_veff_diag[A, A] = np.einsum("tsuv, uv -> ts", vxc_diag[:, :, sA], D[sA]) * 2

### 异核部分

In [23]:
partial_hess_pyscf = scf_hess.partial_hess_elec()

In [24]:
from pyscf.dft import numint
import time

In [25]:
vxc_deriv2_pyscf = hessian.rks._get_vxc_deriv2(scf_hess, C, mo_occ, 16000)
hess_deriv2_pyscf = np.zeros((natm, natm, 3, 3))
for A in range(natm):
    for B in range(natm):
        sA, sB = mol_slice(A), mol_slice(B)
        hess_deriv2_pyscf[A, B] = np.einsum("tsuv, uv -> ts", vxc_deriv2_pyscf[A, :, :, sB], D[sB]) * 2

### Time

In [26]:
# %%time

#---
time0 = time.time();
vmat = np.zeros((mol.natm,3,3,nao,nao))
dR_rho1 = np.zeros((3, 4, ngrid))
dR_rho1_0  = 2 * np.einsum("tgk , gl , li -> tgki ", grid_ao_1, grid_ao_0, Co, optimize=True)
dR_rho1_1  = 2 * np.einsum("tsgk, gl , li -> tsgki", grid_ao_2, grid_ao_0, Co, optimize=True)
dR_rho1_1 += 2 * np.einsum("tgk , sgl, li -> tsgki", grid_ao_1, grid_ao_1, Co, optimize=True)
print(time.time() - time0)

#---
time0 = time.time();
wv_0 = np.zeros((3, ngrid, nao, nocc))
wv_1 = np.zeros((3, 3, ngrid, nao, nocc))
wv_0 += .5* np.einsum("g, g, tgki          -> tgki ", grid_weight, grid_frr, dR_rho1_0,                         optimize=True)
wv_1 += 2 * np.einsum("g, g, tgki , rg     -> trgki", grid_weight, grid_frg, dR_rho1_0, grid_rho_1,             optimize=True)
wv_0 +=     np.einsum("g, g, trgki, rg     -> tgki ", grid_weight, grid_frg, dR_rho1_1, grid_rho_1,             optimize=True)
wv_1 += 4 * np.einsum("g, g, tsgki, sg, rg -> trgki", grid_weight, grid_fgg, dR_rho1_1, grid_rho_1, grid_rho_1, optimize=True)
wv_1 += 2 * np.einsum("g, g, trgki         -> trgki", grid_weight, grid_fg,  dR_rho1_1,                         optimize=True)
print(time.time() - time0)

#---
time0 = time.time();
vmat  = np.einsum("tgki,  sgu,  gv , vj -> tskuij", wv_0, grid_ao_1, grid_ao_0, Co, optimize=True)
vmat += np.einsum("tgki,  gv,   sgu, vj -> tskuij", wv_0, grid_ao_0, grid_ao_1, Co, optimize=True)
vmat += np.einsum("trgki, srgu, gv , vj -> tskuij", wv_1, grid_ao_2, grid_ao_0, Co, optimize=True)
vmat += np.einsum("trgki, rgv,  sgu, vj -> tskuij", wv_1, grid_ao_1, grid_ao_1, Co, optimize=True)
print(time.time() - time0)

#---
time0 = time.time();
hess_vmat_tmp = np.zeros((natm, natm, 3, 3))
for A in range(natm):
    for B in range(natm):
        sA, sB = mol_slice(A), mol_slice(B)
        hess_vmat_tmp[A, B] = 8 * np.einsum("tskuij, ki, uj -> ts", vmat[:, :, sA, sB], Co[sA], Co[sB])
print(time.time() - time0)

3.644152879714966
9.904576301574707
10.93254017829895
0.004670143127441406


In [53]:
timeA = timeB = timeC = 0
hess_deriv2 = np.zeros((natm, natm, 3, 3))

for A in range(natm):
    sA = mol_slice(A)

    #---
    time0 = time.time();
    grid_A_rho_1  = 2 * np.einsum("tgk , gl , kl -> tg ", grid_ao_1[:, :, sA],    grid_ao_0, D[sA], optimize=True)
    grid_A_rho_2  = 2 * np.einsum("trgk, gl , kl -> trg", grid_ao_2[:, :, :, sA], grid_ao_0, D[sA], optimize=True)
    grid_A_rho_2 += 2 * np.einsum("tgk , rgl, kl -> trg", grid_ao_1[:, :, sA],    grid_ao_1, D[sA], optimize=True)
    timeA += time.time() - time0

    #---
    time0 = time.time();
    wv_0 = np.zeros((3, ngrid))
    wv_1 = np.zeros((3, 3, ngrid))
    wv_0 += .5* np.einsum("g, g, tg          -> tg ", grid_weight, grid_frr, grid_A_rho_1,                         optimize=True)
    wv_1 += 2 * np.einsum("g, g, tg , rg     -> trg", grid_weight, grid_frg, grid_A_rho_1, grid_rho_1,             optimize=True)
    wv_0 +=     np.einsum("g, g, trg, rg     -> tg ", grid_weight, grid_frg, grid_A_rho_2, grid_rho_1,             optimize=True)
    wv_1 += 4 * np.einsum("g, g, twg, rg, wg -> trg", grid_weight, grid_fgg, grid_A_rho_2, grid_rho_1, grid_rho_1, optimize=True)
    wv_1 += 2 * np.einsum("g, g, trg         -> trg", grid_weight, grid_fg,  grid_A_rho_2,                         optimize=True)
    timeB += time.time() - time0

    #---
    time0 = time.time();
    for B in range(natm):
        sB = mol_slice(B)
        hess_deriv2[A, B]  = 4 * np.einsum("tg,  sgu,  gv , uv -> ts", wv_0, grid_ao_1[:, :, sB],    grid_ao_0, D[sB], optimize=True) 
        hess_deriv2[A, B] += 2 * np.einsum("trg, srgu, gv , uv -> ts", wv_1, grid_ao_2[:, :, :, sB], grid_ao_0, D[sB], optimize=True)
        hess_deriv2[A, B] += 2 * np.einsum("trg, sgu,  rgv, uv -> ts", wv_1, grid_ao_1[:, :, sB],    grid_ao_1, D[sB], optimize=True)
        hess_deriv2[B, A] = hess_deriv2[A, B].T
    timeC += time.time() - time0

#---
        
print(timeA)
print(timeB)
print(timeC)

0.3527677059173584
0.12494540214538574
1.4512510299682617


In [54]:
ipip  = 0.5 * np.einsum("g, g, sgu, tgv -> tsuv", grid_weight, grid_fr, grid_ao_1, grid_ao_1, optimize=True)
ipip += 2 * np.einsum("g, g, rg, rsgu, tgv -> tsuv", grid_weight, grid_fg, grid_rho_1, grid_ao_2, grid_ao_1, optimize=True)

for A in range(natm):
    for B in range(natm):
        sA, sB = mol_slice(A), mol_slice(B)
        hess_deriv2[A, B] += np.einsum("tsuv, uv -> ts", ipip[:, :, sB, sA], D[sB, sA]) * 2
        hess_deriv2[A, B] += np.einsum("stvu, uv -> ts", ipip[:, :, sA, sB], D[sB, sA]) * 2
np.allclose(hess_deriv2, hess_deriv2_pyscf)

True

In [49]:
hess_deriv2 - hess_deriv2_pyscf

array([[[[ 0.     ,  0.     , -0.     ],
         [-0.     ,  0.     , -0.     ],
         [-0.     ,  0.     ,  0.     ]],

        [[-0.     , -0.00236,  0.03204],
         [ 0.00236,  0.     ,  0.03204],
         [-0.03204, -0.03204, -0.     ]],

        [[-0.     , -0.00206, -0.022  ],
         [ 0.00206,  0.     ,  0.00012],
         [ 0.022  , -0.00012,  0.     ]],

        [[-0.     ,  0.00128,  0.00187],
         [-0.00128,  0.     , -0.00794],
         [-0.00187,  0.00794, -0.     ]]],


       [[[-0.     , -0.     ,  0.     ],
         [ 0.     ,  0.     ,  0.     ],
         [-0.     ,  0.     , -0.     ]],

        [[ 0.     , -0.     ,  0.     ],
         [ 0.     ,  0.     ,  0.     ],
         [-0.     , -0.     ,  0.     ]],

        [[-0.     , -0.00128,  0.00794],
         [ 0.00128,  0.     , -0.00187],
         [-0.00794,  0.00187, -0.     ]],

        [[-0.     ,  0.00206, -0.00012],
         [-0.00206,  0.     ,  0.022  ],
         [ 0.00012, -0.022  ,  0.     ]]]

In [32]:
def get_hess_ao_noU_hcore(A, B):
    ao_matrix = np.zeros((3 * 3, nao, nao))
    zA, zB = mol.atom_charge(A), mol.atom_charge(B)
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        ao_matrix[:, sA] += int1e_ipipkin[:, sA]
        ao_matrix[:, sA] += int1e_ipipnuc[:, sA]
        with mol.with_rinv_as_nucleus(A):
            ao_matrix -= zA * mol.intor('int1e_ipiprinv')
            ao_matrix -= zA * mol.intor('int1e_iprinvip')
    ao_matrix[:, sA, sB] += int1e_ipkinip[:, sA, sB]
    ao_matrix[:, sA, sB] += int1e_ipnucip[:, sA, sB]
    with mol.with_rinv_as_nucleus(B):
        ao_matrix[:, sA] += zB * mol.intor('int1e_ipiprinv')[:, sA]
        ao_matrix[:, sA] += zB * mol.intor('int1e_iprinvip')[:, sA]
    with mol.with_rinv_as_nucleus(A):
        ao_matrix[:, sB] += zA * mol.intor('int1e_ipiprinv')[:, sB]
        ao_matrix[:, sB] += zA * mol.intor('int1e_iprinvip').swapaxes(1, 2)[:, sB]
    ao_matrix += ao_matrix.swapaxes(1, 2)
    return ao_matrix

eri_mat1 = np.einsum("Tuvkl, kl -> Tuv", int2e_ipip1, D) * 2 - c_x * np.einsum("Tukvl, kl -> Tuv", int2e_ipip1, D)
eri_mat2 = np.einsum("Tuvkl, kl -> Tuv", int2e_ipvip1, D) * 2 - c_x * np.einsum("Tukvl, kl -> Tuv", int2e_ip1ip2, D)
eri_tensor1 = int2e_ip1ip2 * 4 - c_x * int2e_ipvip1.swapaxes(2, 3) - c_x * int2e_ip1ip2.swapaxes(2, 4)

def get_hess_ao_noU_eri(A, B):
    ao_matrix = np.zeros((9, nao, nao))
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        ao_matrix[:, sA] += eri_mat1[:, sA]
    ao_matrix[:, sA, sB] += eri_mat2[:, sA, sB]
    ao_matrix[:, sA] += np.einsum("Tuvkl, kl -> Tuv", eri_tensor1[:, sA, :, sB], D[sB])
    return ao_matrix

def get_hess_ao_noU_S(A, B):
    ao_matrix = np.zeros((9, nao, nao))
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        ao_matrix[:, sA] -= int1e_ipipovlp[:, sA] * 2
    ao_matrix[:, sA, sB] -= int1e_ipovlpip[:, sA, sB] * 2
    return ao_matrix

def get_hess_noU(A, B):
    return (np.einsum("Tuv, uv -> T", get_hess_ao_noU_hcore(A, B) + get_hess_ao_noU_eri(A, B), D).reshape(3, 3)
            + np.einsum("Tuv, uv -> T",  + get_hess_ao_noU_S(A, B), De).reshape(3, 3))

hess_noU_noGGA = np.array([ [ get_hess_noU(A, B) for B in range(natm) ] for A in range(natm) ])

In [33]:
np.allclose(
    hess_noU_noGGA + hess_veff_diag + hess_deriv2,
    partial_hess_pyscf
)

True

In [ ]:
def get_hess_ao_noU_hcore(A, B):
    ao_matrix = np.zeros((3 * 3, nao, nao))
    zA, zB = mol.atom_charge(A), mol.atom_charge(B)
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        ao_matrix[:, sA] += int1e_ipipkin[:, sA]
        ao_matrix[:, sA] += int1e_ipipnuc[:, sA]
        with mol.with_rinv_as_nucleus(A):
            ao_matrix -= zA * mol.intor('int1e_ipiprinv')
            ao_matrix -= zA * mol.intor('int1e_iprinvip')
    ao_matrix[:, sA, sB] += int1e_ipkinip[:, sA, sB]
    ao_matrix[:, sA, sB] += int1e_ipnucip[:, sA, sB]
    with mol.with_rinv_as_nucleus(B):
        ao_matrix[:, sA] += zB * mol.intor('int1e_ipiprinv')[:, sA]
        ao_matrix[:, sA] += zB * mol.intor('int1e_iprinvip')[:, sA]
    with mol.with_rinv_as_nucleus(A):
        ao_matrix[:, sB] += zA * mol.intor('int1e_ipiprinv')[:, sB]
        ao_matrix[:, sB] += zA * mol.intor('int1e_iprinvip').swapaxes(1, 2)[:, sB]
    ao_matrix += ao_matrix.swapaxes(1, 2)
    return ao_matrix

eri_mat1 = np.einsum("Tuvkl, kl -> Tuv", int2e_ipip1, D) * 2 - c_x * np.einsum("Tukvl, kl -> Tuv", int2e_ipip1, D)
eri_mat2 = np.einsum("Tuvkl, kl -> Tuv", int2e_ipvip1, D) * 2 - c_x * np.einsum("Tukvl, kl -> Tuv", int2e_ip1ip2, D)
eri_tensor1 = int2e_ip1ip2 * 4 - c_x * int2e_ipvip1.swapaxes(2, 3) - c_x * int2e_ip1ip2.swapaxes(2, 4)

def get_hess_ao_noU_eri(A, B):
    ao_matrix = np.zeros((9, nao, nao))
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        ao_matrix[:, sA] += eri_mat1[:, sA]
    ao_matrix[:, sA, sB] += eri_mat2[:, sA, sB]
    ao_matrix[:, sA] += np.einsum("Tuvkl, kl -> Tuv", eri_tensor1[:, sA, :, sB], D[sB])
    return ao_matrix

def get_hess_ao_noU_S(A, B):
    ao_matrix = np.zeros((9, nao, nao))
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        ao_matrix[:, sA] -= int1e_ipipovlp[:, sA] * 2
    ao_matrix[:, sA, sB] -= int1e_ipovlpip[:, sA, sB] * 2
    return ao_matrix

def get_hess_noU(A, B):
    return (np.einsum("Tuv, uv -> T", get_hess_ao_noU_hcore(A, B) + get_hess_ao_noU_eri(A, B), D).reshape(3, 3)
            + np.einsum("Tuv, uv -> T",  + get_hess_ao_noU_S(A, B), De).reshape(3, 3))

hess_noU_noGGA = np.array([ [ get_hess_noU(A, B) for B in range(natm) ] for A in range(natm) ])

In [ ]:
np.allclose(
    hess_noU_noGGA + hess_veff_diag + hess_deriv2_pyscf,
    scf_hess.partial_hess_elec()
)

## TMP

In [ ]:
%%time

vmat = np.zeros((mol.natm,3,3,nao,nao))
ipip = np.zeros((3,3,nao,nao))

ni = scf_eng._numint
xctype = ni._xc_type(scf_eng.xc)
aoslices = mol.aoslice_by_atom()
shls_slice = (0, mol.nbas)
ao_loc = mol.ao_loc_nr()

# grid_rho: rho = ni.eval_rho2(mol, ao[:4], mo_coeff, mo_occ, mask, 'GGA')
# grid_vxc, grid_fxc: vxc, fxc = ni.eval_xc(mf.xc, rho, 0, deriv=2)[1:3]

#--------------------------------------------------------------------------

#-- wv = numint._rks_gga_wv0(grid_rho, grid_vxc, grid_weight)

# wv = np.zeros((4, ngrid))
# wv[0] = 0.5 * np.einsum("g, g -> g", grid_weight, grid_fr)
# wv[1:4] = 2 * np.einsum("g, g, rg -> rg", grid_weight, grid_fg, grid_rho_1)

#--------------------------------------------------------------------------

#-- aow = grad.rks._make_dR_dao_w(grid_ao, wv)

# aow = np.einsum("tgu, g -> tgu", grid_ao_1, wv[0])
# aow += np.einsum("trgu, rg -> tgu", grid_ao_2, wv[1:4])

#--------------------------------------------------------------------------

aow = 0.5 * np.einsum("g, g, sgv -> sgv", grid_weight, grid_fr, grid_ao_1, optimize=True)
aow += 2 * np.einsum("g, g, rg, rsgv -> sgv", grid_weight, grid_fg, grid_rho_1, grid_ao_2, optimize=True)

#==========================================================================

#-- hessian.rks._d1d2_dot_(ipip, mol, aow, grid_ao[1:4], grid_mask, ao_loc, False)
# ipip = np.einsum("sgu, tgv -> tsuv", aow, grid_ao_1)

ipip += 0.5 * np.einsum("g, g, sgu, tgv -> tsuv", grid_weight, grid_fr, grid_ao_1, grid_ao_1, optimize=True)
ipip += 2 * np.einsum("g, g, rg, rsgu, tgv -> tsuv", grid_weight, grid_fg, grid_rho_1, grid_ao_2, grid_ao_1, optimize=True)

#--------------------------------------------------------------------------

#-- ao_dm0 = [numint._dot_ao_dm(mol, grid_ao[i], D, grid_mask, shls_slice, ao_loc)
#--           for i in range(4)]

ao_dm0 = np.einsum("uv, sgv -> sgu", D, grid_ao[0:4])

#--------------------------------------------------------------------------

for ia in range(natm):
    sA = mol_slice(ia)
#--------------------------------------------------------------------------
    #--- wv = dR_rho1 = hessian.rks._make_dR_rho1(grid_ao, ao_dm0, ia, aoslices)
    wv = np.zeros((3, 4, ngrid))
    dR_rho1 = np.zeros((3, 4, ngrid))
    dR_rho1[:, 0] = 2 * np.einsum("tgu, uv, gv -> tg", grid_ao_1[:, :, sA], D[sA], grid_ao_0, optimize=True)
    dR_rho1[:, 1:4] = 2 * np.einsum("tsgu, uv, gv -> tsg", grid_ao_2[:, :, :, sA], D[sA], grid_ao_0, optimize=True)
    dR_rho1[:, 1:4] += 2 * np.einsum("tgu, uv, sgv -> tsg", grid_ao_1[:, :, sA], D[sA], grid_ao_1, optimize=True)
#--------------------------------------------------------------------------
    #--- wv[0] = numint._rks_gga_wv1(grid_rho, dR_rho1[0], grid_vxc, grid_fxc, grid_weight)
    #--- wv[1] = numint._rks_gga_wv1(grid_rho, dR_rho1[1], grid_vxc, grid_fxc, grid_weight)
    #--- wv[2] = numint._rks_gga_wv1(grid_rho, dR_rho1[2], grid_vxc, grid_fxc, grid_weight)
    
    # wv[0, 0] = 0.5 * np.einsum("g, g, g -> g", grid_weight, grid_frr, dR_rho1[0, 0], optimize=True)
    # wv[0, 0] += np.einsum("g, g, rg, rg -> g", grid_weight, grid_frg, dR_rho1[0, 1:4], grid_rho_1, optimize=True)
    
    # wv[0, 1:4] = 2 * np.einsum("g, g, g, sg -> sg", grid_weight, grid_frg, dR_rho1[0, 0], grid_rho_1, optimize=True)
    # wv[0, 1:4] += 4 * np.einsum("g, g, rg, rg, sg -> sg", grid_weight, grid_fgg, grid_rho_1, dR_rho1[0, 1:4], grid_rho_1, optimize=True)
    # wv[0, 1:4] += 2 * np.einsum("g, g, sg -> sg", grid_weight, grid_fg, dR_rho1[0, 1:4], optimize=True)
    
    wv[:, 0] = 0.5 * np.einsum("g, g, tg -> tg", grid_weight, grid_frr, dR_rho1[:, 0], optimize=True)
    wv[:, 0] += np.einsum("g, g, trg, rg -> tg", grid_weight, grid_frg, dR_rho1[:, 1:4], grid_rho_1, optimize=True)
    wv[:, 1:4] = 2 * np.einsum("g, g, tg, sg -> tsg", grid_weight, grid_frg, dR_rho1[:, 0], grid_rho_1, optimize=True)
    wv[:, 1:4] += 4 * np.einsum("g, g, rg, trg, sg -> tsg", grid_weight, grid_fgg, grid_rho_1, dR_rho1[:, 1:4], grid_rho_1, optimize=True)
    wv[:, 1:4] += 2 * np.einsum("g, g, tsg -> tsg", grid_weight, grid_fg, dR_rho1[:, 1:4], optimize=True)
#--------------------------------------------------------------------------
    #--- aow = grad.rks._make_dR_dao_w(grid_ao, wv[0])
    #--- grad.rks._d1_dot_(vmat[ia,0], mol, aow, grid_ao[0], grid_mask, ao_loc, True)
    #--- aow = grad.rks._make_dR_dao_w(grid_ao, wv[1])
    #--- grad.rks._d1_dot_(vmat[ia,1], mol, aow, grid_ao[0], grid_mask, ao_loc, True)
    #--- aow = grad.rks._make_dR_dao_w(grid_ao, wv[2])
    #--- grad.rks._d1_dot_(vmat[ia,2], mol, aow, grid_ao[0], grid_mask, ao_loc, True)
    
    # print(vmat[ia, 0])
    # print(vmat[ia, 0].shape)
    # tmp = np.zeros((3, nao, nao))
    # tmp += np.einsum("g, sgu, gv -> suv", wv[0, 0], grid_ao_1, grid_ao_0)
    # tmp += np.einsum("rg, srgu, gv -> suv", wv[0, 1:4], grid_ao_2, grid_ao_0)
    # print(np.allclose(tmp, vmat[ia, 0]))
    
    vmat[ia] += np.einsum("tg, sgu, gv -> tsuv", wv[:, 0], grid_ao_1, grid_ao_0, optimize=True)
    vmat[ia] += np.einsum("trg, srgu, gv -> tsuv", wv[:, 1:4], grid_ao_2, grid_ao_0, optimize=True)
#--------------------------------------------------------------------------
    #--- aow = np.einsum('npi,Xnp->Xpi', grid_ao[:4], wv)
    #--- hessian.rks._d1d2_dot_(vmat[ia], mol, grid_ao[1:4], aow, grid_mask, ao_loc, False)
    vmat[ia] += np.einsum("trg, rgv, sgu -> tsuv", wv, grid_ao[:4], grid_ao_1, optimize=True)
#--------------------------------------------------------------------------
ao_dm0 = aow = None

for ia in range(mol.natm):
    p0, p1 = aoslices[ia][2:]
    vmat[ia,:,:,:,p0:p1] += ipip[:,:,:,p0:p1]
    vmat[ia,:,:,:,p0:p1] += ipip[:,:,p0:p1].transpose(1,0,3,2)
    
print(np.allclose(vmat, vxc_deriv2_pyscf))